In [1]:
def readGenome(filename):
    fullGenome = ""
    with open(filename, 'r') as fh:
        for line in fh:
            if not (line[0] == '>'):
                fullGenome += line.rstrip()
    return fullGenome

genome = readGenome('Wk3-Data/chr1.GRCh38.excerpt.fasta')

In [2]:
def editDistanceWithPlaceSelection(word1: str, word2: str) -> int:
        row = len(word1)
        col = len(word2)
        if row == 0:
            return col
        if col == 0:
            return row
        dp = [[0] * (col + 1) for _ in range(row + 1)]
        for i in range(row + 1):
            dp[i][0] = i
        for i in range(col + 1):
            dp[0][i] = 0
        for i in range(1, row + 1):
            for j in range(1, col + 1):
                if word1[i - 1] == word2[j - 1]:
                    replace = dp[i - 1][j - 1]
                else:
                    replace = dp[i - 1][j - 1] + 1
                dp[i][j] = min(dp[i - 1][j] + 1, dp[i][j - 1] + 1, replace)
        minNum = dp[row][0]
        for i in range(col + 1):
            if dp[row][i] < minNum:
                minNum = dp[row][i]
        return minNum

In [3]:
# Question 1
p = 'GCTGATCGATCGTACG'
editDistanceWithPlaceSelection(p, genome)

3

In [47]:
# Question 2
p = 'GATTTACCAGATTGAG'
editDistanceWithPlaceSelection(p, genome)

2

In [5]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()
            seq = fh.readline().rstrip()
            fh.readline()
            qua = fh.readline().rstrip()
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qua)
    return sequences, qualities

In [108]:
reads, _ = readFastq("Wk3-Data/ERR266411_1.for_asm.fastq")

In [100]:
def buildSetsByKmer(reads, k = 30):
    kmersLocation = {}
    for i in range(len(reads)):
        seq = reads[i]
        for j in range(len(seq) - k + 1):
            kmer = seq[j:j + k]
            if kmer in kmersLocation:
                idxList = kmersLocation.get(kmer)
                idxList.add(i)
                kmersLocation[kmer] = idxList
            else:
                kmersLocation[kmer] = set([i])
    return kmersLocation

In [109]:
KmerSet = buildSetsByKmer(reads, 30)

In [102]:
def overlap(first, second, min_len = 3):
    start = 0
    while (start < len(first) - min_len + 1):
        start = first.find(second[:min_len], start)
        if (start == -1):
            return 0
        if second.startswith(first[start:]):
            return len(first) - start
        start += 1

In [103]:
def OverlapMap(reads, k, kmersLoc):
    overLaps = {}
    for i in range(len(reads)):
        seq = reads[i]
        sufx = seq[-k:]
        for j in kmersLoc[sufx]:
            if i == j:
                continue
            t = overlap(reads[i], reads[j], k)
            if t > 0:
                overLaps[(i, j)] = t
    return overLaps

In [110]:
# Question 3, 4
ans = OverlapMap(reads, 30, KmerSet)
print(len(ans))
distinctFirstNode = set()
for (i, j), k in ans.items():
    distinctFirstNode.add(i)
print(len(distinctFirstNode))

904746
7161


In [105]:
def naiveOverlapMap(reads, k):
    overLaps = {}
    for a, b in list(permutations(reads, 2)):
        olen = overlap(a, b, k)
        if olen > 0:
            overLaps[(a, b)] = olen
    return overLaps

In [106]:
len(naiveOverlapMap(reads, 30))

1130